In [ ]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import sqlite3 as sql
import numpy as np
import datetime as datetime
from io import BytesIO
import xlsxwriter
from dataiku.core.sql import SQLExecutor2
import time
from zipfile import Zipfile
from os.path import basename
import os
import shutil
import re

# storing proj kep in variable
client = dataiku.api_client()
proj = client.get_default_project()
project = proj.get_summary()['projectKey']

# Read recipe inputs
mydataset = dataiku.Dataset("Table name")
mydataset_df = mydataset.get_dataframe()

#-------Or----------
mydataset = dataiku.Dataset("Table name")
executor = SQLExecutor2(dataset = mydataset)
tb_tracker_df = executor.query_to_df(
"""
select "company code", "year" from "AN@LYTIC_PG_PR...%s" """  %(project) + """."%s_table name" 
where "year" in (CAST((Year(GETDATE())) AS int), CAST((Year(GETDATE())-1) AS int))
group by "company code", "year"
""" %(project))

# appending dfs to create one and drop duplicate values
frames = [frame1, frame2, frame3]
compcd = pd.concat(frames).drop_duplicates()

# checking for user inputs
input_var = dataiku.get_custom_variables()['Variable name']
elements = input_var.split(',')
user_input = []
for i in elements:
    if len(i.strip())>0:
        user_input.append(i.strip())
        
# filtering dataset based on user inputs
if len(''.join(user_input))>0:
    mydataset_df_selected = mydataset_df[mydataset_df['Col Name'].isin(user_input)]
else:
    mydataset_df_selected = mydataset_df
    
# initializing counter variables
i = 0
j = 1
# Declaring Sharepoint output folder
out_folder = dataiku.folder("xxyyzzaabb")

# Querying the tables with company code and year combo

for comp, year_df in enumerate(mydataset_df_selected.values):
    mydataset = dataiku.Dataset(" Table name")
    executor = SQLExecutor2(dataset = mydataset)
    sqlstring = ("""select * from "AN@LYTICS_PG_PR_%s" """ %(project) + """."%s_Table name" where""" %(project) + """ \"COMPANY CODE\" = %d""" %(year_df[0]) + """ and \"YEAR\" = %d""" %(year_df[1]))
    Table_df = executor.query_to_df(sqlstring)

    mydataset1 = dataiku.Dataset("Table name 1")
    executor = SQLExecutor2(dataset = mydataset1)
    sqlstring = ("""select * from "AN@LYTICS_PG_PR_%s" """ %(project) + """."%s_Table name1" where""" %(project) + """ \"COMPANY CODE\" = %d""" %(year_df[0]) + """ and \"YEAR\" = %d""" %(year_df[1]))
    Table_df1 = executor.query_to_df(sqlstring)

    mydataset2 = dataiku.Dataset("Table name 2")
    executor = SQLExecutor2(dataset = mydataset2)
    sqlstring = ("""select * from "AN@LYTICS_PG_PR_%s" """ %(project) + """."%s_Table name2" where""" %(project) + """ \"COMPANY CODE\" = %d""" %(year_df[0]) + """ and \"YEAR\" = %d""" %(year_df[1]))
    Table_df2 = executor.query_to_df(sqlstring)

    company_code = year_df[0]
    fisc@lyear_current = year_df[1]
    
    # Storing the dataframes in a dictionary
    frames = {}
    if Table_df.empty:
        pass
    else:
        col_names = ['col1', 'col2', '....', 'colX']
        table_df = Table_df[col_names].copy()
        frames['Table_df'] = table_df
        
    if Table_df1.empty:
        pass
    else:
        col_names = ['col1', 'col2', '....', 'colX']
        table_df1 = Table_df1[col_names].copy()
        frames['Table_df1'] = table_df1
        
    if Table_df2.empty:
        pass
    else:
        col_names = ['col1', 'col2', '....', 'colX']
        table_df2 = Table_df2[col_names].copy()
# Iterating through the dataframe 5,00,000 rows at a time
        CHUNK_SIZE = 500000
        iterator =1
        for chunk_num in range(len(table_df2) // CHUNK_SIZE +1):
            start_index = chunk_num*CHUNK_SIZE
            end_index = min(chunk_num*CHUNK_SIZE + CHUNK_SIZE, len(table_df2))
            chunk = table_df2[start_index:end_index]
            frames['Table_df2' + str(iterator)] = chunk
            iterator += 1

# loop to iterate through the dict and create formatted excel output
    stream = BytesIO()
    excel_writer = pd.ExcelWriter(stream, engine = 'xlsxwriter')
    
    for keys, filter_df in frames.items():
        
        filter_df.to_excel(excel_writer, sheet_name = keys, index = False)
        workbook = excel_writer.book
        worksheet = excel_writer.sheets[keys]
        
        # formatting the excel
        header_format = workbook.add_format({'bg_color' : '#e6e6e6',
                                            'font_color': 'black',
                                            'font_size': 'Arial',
                                            'bold': True})
        
        right_header_format = workbook.add_format({'bg_color' : '#e6e6e6',
                                            'font_color': 'black',
                                            'font_size': 'Arial',
                                            'align' : 'right',
                                            'bold': True})
        number_format = workbook.add_format({'num_format': '###,###,###,###,###0'})
        number_rows = len(filter_df.index) + 1
        for col_header_num, header_value in enumerate(filter_df.columns.values):
            if (keys == 'Table_df'):
                num_header1 = ['Col1', 'Col2' , '...', 'colX']
                if (header_value in num_header1):
                    worksheet.write(0, col_header_num, header_value, right_header_format)
                else:
                    workshet.write(0, col_header_num, header_value, header_format)
            else (keys in LocalOnly_sheet_names):
                if(col_header_num == 26):
                    worksheet.write(0, col_header_num, header_value, right_header_format)
                else:
                    worksheet.write(0, col_header_num, header_value, header_format)
                    
        if (keys == 'Table_df'):
            worksheet.conditional_format("$T$2:$Z%d" % (number_rows),
                                         {'type': 'cell',
                                         'criteria': '!=',
                                         'value': 0,
                                         "format": number_format})
        if (keys.startswith('Table_df2')):
            worksheet.conditional_format("$C$2:$P%d" % (number_rows),
                                         {'type': 'cell',
                                         'criteria': '!=',
                                         'value': 0,
                                         "format": number_format})            
            
        measurer = np.vectorize(len)
        for i in range(filter_df.shepe[1]):
            # find width of i column
            columnNamewidth = len(filter_df.columns[i])
            # find max width of values in i column
            maxColumnWidth = measurer(filter_df.iloc[:, i].values.astype(str)).max(axis = 0)
            # set column width only for that column, +2 for padding
            worksheet.set_column(i, i, max(columnNameWidth, maxColumnWidth)+2)
    excel_writer.save()
    stream.seek(0)
    # End of for loop
    
    date_stamp = dt.datetime.today().strftime('%Y-%m-%d')
    with out_folder.get_writer("%s/%s/%d/Folder_Name/%s_%d_Table Name_%s.xlsx" %(currcountry, company, fisc@lyrcurr, company,fisc@lyrcurr, date_stamp)) as writer:
        successful = False
        attempts = 0
        while not successful and attempts < 3:
            attempts += 1
            try:
                writer.writer(stream.read())
                successful = True
            except:
                time.sleep(5)

'''
# Reading the local folder to zip all the files
files = out_folder.list_paths_in_partition()
filesIndex = 0
filesNameIndex = 1
files2 = files.copy()

#zipping the contents of the folder
with ZipFile(out_folder.get_path()+ '/Name of the file.zip', 'w', zipfile.ZIP_DEFLATED) as zipobj:
    while(filesIndex < len(files2)):
        zipObj.write(out_folder.get_path() + files2[filesIndex], files2[filesIndex])
        filesIndex += 1
        
# Copying the local folder to Share point
client = dataiku.api_client()
project = client.get_project(dataiku.default_project_key())

source_folder = project.get_managed_folder("zzxxxccvv")
target_folder = project.get_managed_folder("SP folder name")

future = source_folder.copy_to(target_folder, write_mode = 'OVERWRITE')
future.wait_for_result()

# Check for zip folder and then move just that
x=0
for paths[x] in paths:
    if re.search("Local Folder", paths[x]):
        with input_folder.get_download_stream(paths[x]) as f:
            data = f.read()
        with output_folder1.get_writer(paths[x]) as w:
            w.write(data)
    x += 1
'''